# Loading Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/drug-classification/drug200.csv')

# Exploring Data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df['Cholesterol'].value_counts()

In [ ]:
df['BP'].value_counts()

In [ ]:
df['Drug'].value_counts()   

# Data Visualization


In [ ]:
import seaborn as sns
sns.swarmplot(data=df,x='Drug',y='Cholesterol')

In [ ]:
sns.swarmplot(data=df,x='Drug',y='Age')

In [ ]:
sns.swarmplot(data=df,x='Drug',y='BP')

# Split Data

In [ ]:
X = df.drop('Drug',axis=1)
y = df['Drug']

In [ ]:
X = pd.get_dummies(X,['Age','BP','Cholesterol'])
y = pd.get_dummies(y,['Drug'])

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

# Building Model 1 -> RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_1 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
model_1.fit(X_train, y_train)

In [ ]:
y_pred = model_1.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

# Building Model 2 -> NN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout ,Flatten
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [ ]:
normalize = Normalization()

In [ ]:
model_2 = Sequential([
    normalize,
    Flatten(input_shape=(9,)),
    Dense(32, activation='tanh'),
    Dropout(0.5),
    Dense(5, activation='sigmoid'),
])

model_2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=10,restore_best_weights=True)

In [ ]:
model_2.fit(x=X_train, 
          y=y_train, 
          epochs=1000,
          batch_size=10,
          validation_data=(X_test, y_test),
           callbacks=[early_stop]
          )

In [ ]:
model_2.evaluate(X_test, y_test)